In [2]:
#install adapter
!pip install azure-storage-blob
!pip install boto3

In [3]:
import json
import boto3
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import StringIO
import pandas as pd
import numpy as np
import requests

In [4]:
# Function

# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [5]:
# Now all_data contains all rows from the API
import requests

url = 'https://data.wa.gov/resource/f6w7-q2d2.json'
offset = 0
limit = 1000  # Maximum number of rows per request

all_data = []

while True:
    params = {'$offset': offset, '$limit': limit}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        all_data.extend(data)

        if len(data) < limit:
            break

        offset += limit
    else:
        print("Error:", response.status_code)
        break


In [6]:
df = pd.DataFrame(all_data)
df.head()

,vin_1_10,county,city,state,zip_code,model_year,make,model,ev_type,cafv_type,electric_range,base_msrp,legislative_district,dol_vehicle_id,geocoded_column,electric_utility,_2020_census_tract,:@computed_region_x4ys_rtnd,:@computed_region_fny7_vc3j,:@computed_region_8ddd_yn5v
0,WAUTPBFF4H,King,Seattle,WA,98126,2017,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16,0,34,235085336,"{'type': 'Point', 'coordinates': [-122.374105,...",CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),53033011500,3009,7,37
1,WAUUPBFF2J,Thurston,Olympia,WA,98502,2018,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16,0,22,237896795,"{'type': 'Point', 'coordinates': [-122.943445,...",PUGET SOUND ENERGY INC,53067011100,2742,10,28
2,5YJSA1E22H,Thurston,Lacey,WA,98516,2017,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,22,154498865,"{'type': 'Point', 'coordinates': [-122.78083, ...",PUGET SOUND ENERGY INC,53067012226,2742,10,28
3,1C4JJXP62M,Thurston,Tenino,WA,98589,2021,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,25,0,20,154525493,"{'type': 'Point', 'coordinates': [-122.85403, ...",PUGET SOUND ENERGY INC,53067012620,2742,10,27
4,5YJ3E1EC9L,Yakima,Yakima,WA,98902,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,14,225996361,"{'type': 'Point', 'coordinates': [-120.524012,...",PACIFICORP,53077000800,2746,4,20


In [7]:
print(df.columns)

Index(['vin_1_10', 'county', 'city', 'state', 'zip_code', 'model_year', 'make',
       'model', 'ev_type', 'cafv_type', 'electric_range', 'base_msrp',
       'legislative_district', 'dol_vehicle_id', 'geocoded_column',
       'electric_utility', '_2020_census_tract', ':@computed_region_x4ys_rtnd',
       ':@computed_region_fny7_vc3j', ':@computed_region_8ddd_yn5v'],
      dtype='object')


In [8]:
# check null data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181458 entries, 0 to 181457
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   vin_1_10                     181458 non-null  object
 1   county                       181455 non-null  object
 2   city                         181455 non-null  object
 3   state                        181458 non-null  object
 4   zip_code                     181455 non-null  object
 5   model_year                   181458 non-null  object
 6   make                         181458 non-null  object
 7   model                        181458 non-null  object
 8   ev_type                      181458 non-null  object
 9   cafv_type                    181458 non-null  object
 10  electric_range               181458 non-null  object
 11  base_msrp                    181458 non-null  object
 12  legislative_district         181060 non-null  object
 13  dol_vehicle_id

In [9]:
# remove rows with missing data
cleaned_df = df.dropna()

In [10]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180373 entries, 0 to 181457
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   vin_1_10                     180373 non-null  object
 1   county                       180373 non-null  object
 2   city                         180373 non-null  object
 3   state                        180373 non-null  object
 4   zip_code                     180373 non-null  object
 5   model_year                   180373 non-null  object
 6   make                         180373 non-null  object
 7   model                        180373 non-null  object
 8   ev_type                      180373 non-null  object
 9   cafv_type                    180373 non-null  object
 10  electric_range               180373 non-null  object
 11  base_msrp                    180373 non-null  object
 12  legislative_district         180373 non-null  object
 13  dol_vehicle_id     

In [11]:
#put into cloud storatge (create and upload)

In [12]:
# Specify the path to your JASON configuration file
config_file_path='config.json'

#Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Print the configuration
# Connection_STRING = config["connectionString"]


CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE="electricalvehicle"
blob_name = "electricalvehicle.csv"
# Conbert DataFrame to CSV
output=StringIO()
cleaned_df.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

Uploaded electricalvehicle.csv to Azure Blob Storage in container electricalvehicle.
